# PTST with non-zero thickness ligament 

To combine axi-symmetric stress state with the fatigue degradation ligament
it is necessary to address the issue of correct representation of the uniform elastic
state.

## Option #1 - application of slide for reduced strain state

To correctly reproduce the axisymmetric strain and stress state, consider the strain tensor
projections
$$
  e_a = n_a \varepsilon_{ab}
$$
Assuming that the axis of symmetry is aligned with $x$ axis,
the normal to the interface is given as $n_a = [0, 1, 0]$. 
Thus, the normal and tangential strain components can be 
detached from the strain tensor by setting
$$
  e_N = \delta_{Na} e_a, e_T = \delta_{Ta} e_a
$$
where
$$
\delta_{Na} = [0, 1, 0], \;
\delta_{Ta} = 
\left[
\begin{array}{lll}
 0, & 1, & 0 \\
 0, & 0, & 1
\end{array}
\right].
$$

$$
  e_N = \varepsilon_{11}, e_T = \varepsilon_{10}.
$$
Note that the strain component $\varepsilon_{00}$ is disregarded. Is it an acceptable assumption? The stress along the ligament is implicitly assumed zero.

$$
\begin{array}{ccc}
 \varepsilon_{00} &  \varepsilon_{T} & 0 \\
 \varepsilon_{T} &  \varepsilon_{N} & 0 \\
 0 &  0 & \varepsilon_{\theta}
\end{array}
$$

$$
\begin{array}{ccc}
 \sigma_{00} &  \sigma_{T} & 0 \\
 \sigma_{T} &  \sigma_{N} & 0 \\
 0 &  0 & \sigma_{\theta}
\end{array}
$$

$$
\newcommand{\sd}[3]{   {#1}_{\mathrm{#2}}^{\mathrm{#3}}
}
\rho \sd{\psi}{T}{}
(\sd{\boldsymbol{u}}{T}{}, \sd{\boldsymbol{u}}{T}{p}, \sd{\omega}{T}{}, z, \boldsymbol{\alpha}) 
=\frac{1}{2} (1 - \sd{\omega}{T}{})  \sd{E}{T}{} \left[ (\sd{\boldsymbol{u}}{T}{} - \sd{\boldsymbol{u}}{T}{p})^T \cdot (\sd{\boldsymbol{u}}{T}{} - \sd{\boldsymbol{u}}{T}{p}) \right]  +  \frac{1}{2} K z^2  + \frac{1}{2} \gamma 
(\boldsymbol{\alpha}^T \cdot \boldsymbol{\alpha}),
$$

$$
\newcommand{\sd}[3]{   {#1}_{\mathrm{#2}}^{\mathrm{#3}}
}
\rho \sd{\psi}{T}{}
(\sd{\boldsymbol{u}}{T}{}, \sd{\boldsymbol{u}}{T}{p}, \sd{\omega}{T}{}, z, \boldsymbol{\alpha}) 
= \frac{1}{2} D_{abcd} \varepsilon_{ab} \varepsilon_{cd}
-
 \frac{1}{2} E_T (\delta_{Ta}n_a \varepsilon_{ab}) (\delta_{Tc}n_c \varepsilon_{cd})
+
 \frac{1}{2} (1- \omega_T) E_T (\delta_{Ta}n_a (\varepsilon_{ab} - n_a n_b \varepsilon^p_T)) (\delta_{Tc}n_c \varepsilon_{cd} - \varepsilon^p_T)
$$

$$
\newcommand{\sd}[3]{   {#1}_{\mathrm{#2}}^{\mathrm{#3}}
}
\rho \sd{\psi}{T}{}
(\sd{\boldsymbol{u}}{T}{}, \sd{\boldsymbol{u}}{T}{p}, \sd{\omega}{T}{}, z, \boldsymbol{\alpha}) 
= \frac{1}{2} D_{abcd} \varepsilon_{ab} \varepsilon_{cd}
-
 \frac{1}{2} D_{abcd} (\delta_{Ta}n_a \varepsilon_{ab}) (\delta_{Tc}n_c \varepsilon_{cd})
+
 \frac{1}{2} (1- \omega_T) D_{abcd} (\delta_{Ta}n_a \varepsilon_{ab} - n_a n_b \varepsilon^p_T) (\delta_{Tc}n_c \varepsilon_{cd} - n_a n_b \varepsilon^p_T) )
$$

$$
 \sigma_{ab} = \frac{\partial \rho \psi_T}{ \partial \varepsilon_{cd} }
$$

$$
\sigma_{ab} = 
 \beta_{abcd}
 D_{cdef}
 \beta_{efgh}
 (\varepsilon_{gh} - \varepsilon_{gh}^p)
$$

In [15]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from bmcs_matmod.slide.vslide_34_TN_axisym import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic

In [42]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                         coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                         coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=37000, nu=0.18)
m2 = MATS3DElastic(E=37000, nu=0.18)

xd12 = XDomainFEInterface(
    integ_factor =  2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [63]:
material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3, 
     E_N=1, S_N=0.5, c_N=3, m=0.05, f_t=2, f_c=60, f_c0=40, eta=0.)
bond_m = Slide34(**material_params)

In [64]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ]
)

In [65]:
lower_fixed_1 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
lower_fixed_0 = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0], var='u', dims=[0], value=0)

upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, -1, :, -1],
         var='u', dims=[1], value=0.00001)

compression_dofs = upper_compression_slice.dofs
compression = -1 / len(compression_dofs)
upper_compression_force_first = [BCDof(var='f', dof=dof, value = compression) 
             for dof in compression_dofs ]
bc1 = [lower_fixed_0, lower_fixed_1] + upper_compression_force_first

In [66]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1

In [67]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



In [68]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()